# Census ACS

Retrieves data from the Census Bureau's American Community Survey 5-year series API for zctas, pumas, and tracts. A specific list of census variables is passed into the script, which are retrieved from the four ACS profile tables. Variables must be retrieved in chunks because only 50 can be passed to the API at a time, and each url varies by geography and retreives them in different combinations. After some processing output is written to a SQLite database. An option to create a metadata table appears at the bottom, but should only be run once for a given extract (acs1 and acs2) and not for each individual geography.

https://www.census.gov/data/developers/data-sets/acs-5year.html

## Variables

In [1]:
import os, requests, json, sqlite3, random, pandas as pd, numpy as np
from IPython.display import clear_output

In [2]:
keyfile='census_key.txt'

#API variables - UPDATE THE YEAR AND GEO
year='2017'
geo='public use microdata area' # 'zip code tabulation area' or 'public use microdata area' or 'tract'
state='36'
dsource='acs'
dsource2='acs5'
dname='profile'

#Variables to read in from spreadsheet - UPDATE WORKSHEET
worksheet='acs1' # 'acs1' or 'acs2'
geoexcelsheet={'zip code tabulation area':'zctas', 'public use microdata area':'pumas', 'tract':'tracts'}
geotype=geoexcelsheet.get(geo)

#SQL output
tabname='{}_{}{}'.format(geotype,year,worksheet)
dbname=os.path.join('outputs','testdb.sqlite')

#Dump files for api data storage
jsonpath=os.path.join('outputs', tabname+'_retrieved_data.json')

## Variable Lists
Get full list of variables from the API, read in our retrieval list, and compare the varianle IDs and names to make sure nothing is missing and that nothing has changed since the last iteration. *Don't move on to the next block until both lists match.* Lastly, read in list of geographies.

In [3]:
datadict={}
dps=['DP02','DP03','DP04','DP05']
for p in dps:
    vars_url = f'https://api.census.gov/data/{year}/{dsource}/{dsource2}/{dname}/groups/{p}.json'
    response=requests.get(vars_url)
    var_data=response.json()
    datadict.update(var_data['variables'])
random.sample(datadict.items(), 2)

[('DP05_0008E',
  {'label': 'Estimate!!SEX AND AGE!!Total population!!15 to 19 years',
   'concept': 'ACS DEMOGRAPHIC AND HOUSING ESTIMATES',
   'predicateType': 'int',
   'group': 'DP05',
   'limit': 0,
   'predicateOnly': True}),
 ('DP05_0082MA',
  {'label': 'Annotation of Margin of Error!!HISPANIC OR LATINO AND RACE!!Total population!!Not Hispanic or Latino!!Some other race alone',
   'predicateType': 'string',
   'group': 'DP05',
   'limit': 0,
   'predicateOnly': True})]

In [4]:
dfexcel = pd.read_excel(os.path.join('inputs','acs_variables.xlsx'),sheet_name=worksheet)
dfexcel.head()

,db_var,census_var,census_label,dtype
0,HSHD01_E,DP02_0001E,Estimate!!HOUSEHOLDS BY TYPE!!Total households,int
1,HSHD01_M,DP02_0001M,Margin of Error!!HOUSEHOLDS BY TYPE!!Total hou...,int
2,HSHD01_PC,DP02_0001PE,Percent Estimate!!HOUSEHOLDS BY TYPE!!Total ho...,int
3,HSHD01_PM,DP02_0001PM,Percent Margin of Error!!HOUSEHOLDS BY TYPE!!T...,int
4,HSHD02_E,DP02_0002E,Estimate!!HOUSEHOLDS BY TYPE!!Total households...,int


In [5]:
dfvars = pd.DataFrame.from_dict(datadict,columns=['label'],orient='index')
dfvars_selected=dfvars.loc[dfvars.index.isin(dfexcel['census_var'])]
dfvars_count=len(dfvars_selected)
dfexcel_count=len(dfexcel['census_var'])

if dfvars_count==dfexcel_count:
    print('There are an equal number of variables in both lists:', dfvars_count)
else:
    print('There is a mismatch in the number of variables; the api has,', dfvars_count, 
          'while the original list has',dfexcel_count,'. Missing:')
    nomatch=dfexcel[~dfexcel['census_var'].isin(dfvars_selected.index)]
    print(nomatch)

There are an equal number of variables in both lists: 248


In [6]:
mismatch=dfexcel[~dfexcel['census_label'].isin(dfvars_selected['label'])]

if len (mismatch) ==0:
    print('All labels match')
else:
    test=pd.merge(mismatch,dfvars_selected, left_on='census_var', right_on=dfvars_selected.index)
    print('These labels do not match:')
    print(test[['census_var','census_label','label']])

All labels match


In [7]:
# Geographic indetifiers: zctas to retrieve, pumas to filter by, and counties containing tracts to retrieve
excelgeo = pd.read_excel(os.path.join('inputs','acs_variables.xlsx'),sheet_name=geotype, dtype=object)
geoids = excelgeo['GEO'].tolist()
print('Number of geographic indetifiers:',len(geoids))

Number of geographic indetifiers: 55


## Retrieve Data
Given the large number of variables in the ACS and limits of the API, variables must be passed to the url in separate blocks or chunks. The first chunk that's captured is written to an empty datalist; the header row and then one row for each geography. Each subsequent chunk is iterated through by row, so each row is appended to the correct row in datalist. In all cases, the last values, identifiers automatically returned with each API call, are not appended.

In [8]:
def chunks(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i+n]

In [9]:
reqvars=list(chunks(dfvars_selected.index.tolist(),46))
reqvars[0].insert(0,'NAME')
reqvars[0].insert(0,'GEO_ID')
print('Number of chunks:',len(reqvars))

Number of chunks: 6


In [10]:
with open(keyfile) as key:
    api_key=key.read().strip()

base_url = f'https://api.census.gov/data/{year}/{dsource}/{dsource2}/{dname}'
base_url

'https://api.census.gov/data/2017/acs/acs5/profile'

In [11]:
#Function for retrieving data; running this block loads it into memory
#Different geographies have different urls, 
#and a different number of identifiers tacked on to the end of each request

def getdata():
    dlist=[]
    for i, v in enumerate(reqvars):
        batchcols=','.join(v)
        if geotype=='zctas':
            data_url = f'{base_url}?get={batchcols}&for={geo}:{g}&key={api_key}'
            dropvar=-1
        elif geotype=='pumas':
            data_url = f'{base_url}?get={batchcols}&for={geo}:*&in=state:{state}&key={api_key}'
            dropvar=-2
        elif geotype=='tracts':
            data_url = f'{base_url}?get={batchcols}&for={geo}:*&in=state:{state}&in=county:{county}&key={api_key}'
            dropvar=-3
        else:
            print('Appropriate geography not specified in variables block')
            break  
        response=requests.get(data_url)
        if response.status_code==200:
            clear_output(wait=True)
            data=response.json()
            for i2, v2 in enumerate(data):
                if i == 0:
                    dlist.append(v2[:dropvar])
                else:
                    for item in v2[:dropvar]:
                        dlist[i2].append(item)
        else:
            print('***Problem with retrieval***, response code',response.status_code)
    return dlist

#### ***THIS BLOCK IS A REQUESTS BLOCK!***
*NOTE: ZCTA retrieval takes a long time - 15 mins for 215 ZCTAs*

In [12]:
#If this block was run successfully for a given table and geography don't rerun - next block pulls from saved json
datalist=[]
if geotype=='zctas':
    for g in geoids:
        georecord=getdata()
        print('Retrieved data for',g)
        if len(datalist)==0:
            datalist.append(georecord[0])
            datalist.append(georecord[1])
        else:
            datalist.append(georecord[1])
elif geotype=='pumas':
    datalist=getdata()
elif geotype=='tracts':
    for county in geoids:
        georecord=getdata()
        print('Retrieved data for',county)
        if len(datalist)==0:
            for tract in georecord:
                datalist.append(tract)
        else:
            for tract in georecord[1:]:
                datalist.append(tract)
    
dlrows=len(datalist)
dlitems=sum(len(x) for x in datalist)
dlbyrow=dlitems / dlrows
print('Retrieved', dlrows, 'records and', dlitems,'data points with', dlbyrow, 'points for each record...')
        
with open(jsonpath, 'w') as f:
    json.dump(datalist, f)
print('Done - Data dumped to json file')

Retrieved 146 records and 36500 data points with 250.0 points for each record...
Done - Data dumped to json file


## Process Data
Replace footnotes with nulls, create a new GEOID2 column, replace census variable names with database variable names.

In [13]:
with open(jsonpath, 'r') as f:
    jsondata=json.load(f)
alldata = pd.DataFrame(jsondata[1:],columns=jsondata[0],dtype=object).rename(columns={
    'GEO_ID':'GEOID','NAME':'GEOLABEL'}).set_index('GEOID')
alldata.info()
# Index and column entries should be 1 row and 1 column less than previous count (excludes header row and index column) 

<class 'pandas.core.frame.DataFrame'>
Index: 145 entries, 7950000US3600200 to 7950000US3604105
Columns: 249 entries, GEOLABEL to DP03_0063PM
dtypes: object(249)
memory usage: 283.2+ KB


In [14]:
alldata.head(3)

,GEOLABEL,DP02_0001E,DP02_0001M,DP02_0001PE,DP02_0001PM,DP02_0002E,DP02_0002M,DP02_0002PE,DP02_0002PM,DP02_0003E,...,DP03_0061PE,DP03_0061PM,DP03_0062E,DP03_0062M,DP03_0062PE,DP03_0062PM,DP03_0063E,DP03_0063M,DP03_0063PE,DP03_0063PM
GEOID,,,,,,,,,,,,,,,,,,,,,
7950000US3600200,"Clinton, Franklin, Essex & Hamilton Counties P...",66988,789,66988,-888888888,42803,904,63.9,1.1,16490,...,2.2,0.3,52613,1176,-888888888,-888888888,65609,1237,-888888888,-888888888
7950000US3604104,"NYC-Queens Community District 11--Bayside, Dou...",43596,456,43596,-888888888,30728,676,70.5,1.4,11536,...,9.6,0.7,78380,3686,-888888888,-888888888,99996,2801,-888888888,-888888888
7950000US3604103,"NYC-Queens Community District 7--Flushing, Mur...",87071,930,87071,-888888888,61338,1171,70.4,1.2,22394,...,5.4,0.5,53236,1707,-888888888,-888888888,74185,1721,-888888888,-888888888


In [15]:
#This is a lousy solution, come up with something better in the future
footnotes=['-999999999','-999999999.0', '-999999999.00',
           '-888888888','-888888888.0', '-888888888.00',
           '-666666666','-666666666.0', '-666666666.00',
           '-555555555','-555555555.0', '-555555555.00',
           '-333333333','-333333333.0', '-333333333.00',
           '-222222222','-222222222.0', '-222222222.00']
alldata.replace(footnotes,np.nan,inplace=True)

In [16]:
idxgeoid2={'zctas':-5, 'pumas':-7,'tracts':-11}
alldata.insert(loc=0, column='GEOID2',value=alldata.index.str[idxgeoid2.get(geotype):])
alldata.head(3)

,GEOID2,GEOLABEL,DP02_0001E,DP02_0001M,DP02_0001PE,DP02_0001PM,DP02_0002E,DP02_0002M,DP02_0002PE,DP02_0002PM,...,DP03_0061PE,DP03_0061PM,DP03_0062E,DP03_0062M,DP03_0062PE,DP03_0062PM,DP03_0063E,DP03_0063M,DP03_0063PE,DP03_0063PM
GEOID,,,,,,,,,,,,,,,,,,,,,
7950000US3600200,3600200,"Clinton, Franklin, Essex & Hamilton Counties P...",66988,789,66988,NaN,42803,904,63.9,1.1,...,2.2,0.3,52613,1176,NaN,NaN,65609,1237,NaN,NaN
7950000US3604104,3604104,"NYC-Queens Community District 11--Bayside, Dou...",43596,456,43596,NaN,30728,676,70.5,1.4,...,9.6,0.7,78380,3686,NaN,NaN,99996,2801,NaN,NaN
7950000US3604103,3604103,"NYC-Queens Community District 7--Flushing, Mur...",87071,930,87071,NaN,61338,1171,70.4,1.2,...,5.4,0.5,53236,1707,NaN,NaN,74185,1721,NaN,NaN


In [17]:
# For PUMAS filter all the geotype for the state by local areas
if geotype == 'pumas':
    acsdata=alldata.loc[alldata.GEOID2.isin(geoids)].copy().astype(object).sort_index()
else:
    acsdata=alldata.copy().astype(object).sort_index()
acsdata.shape

(55, 250)

In [18]:
#Dictionary of column names from the census and the nyc geodatabase
cv_to_db=dict(zip(dfexcel.census_var, dfexcel.db_var))
random.sample(cv_to_db.items(), 5)

[('DP02_0056PM', 'SCH05_PM'),
 ('DP02_0062PE', 'EDU05_PC'),
 ('DP03_0001E', 'EMP01_E'),
 ('DP03_0001PE', 'EMP01_PC'),
 ('DP02_0059E', 'EDU02_E')]

In [19]:
#Rename the census variables to nyc geodatabase variables
acsdata.rename(columns=cv_to_db,inplace=True)
acsdata.head(3)

,GEOID2,GEOLABEL,HSHD01_E,HSHD01_M,HSHD01_PC,HSHD01_PM,HSHD02_E,HSHD02_M,HSHD02_PC,HSHD02_PM,...,INC11_PC,INC11_PM,INC12_E,INC12_M,INC12_PC,INC12_PM,INC13_E,INC13_M,INC13_PC,INC13_PM
GEOID,,,,,,,,,,,,,,,,,,,,,
7950000US3603701,3603701,"NYC-Bronx Community District 8--Riverdale, Fie...",42374,536,42374,NaN,25588,848,60.4,1.8,...,6.5,0.8,60118,2785,NaN,NaN,83887,3186,NaN,NaN
7950000US3603702,3603702,"NYC-Bronx Community District 12--Wakefield, Wi...",48495,538,48495,NaN,33732,802,69.6,1.5,...,3.2,0.6,50526,1775,NaN,NaN,65905,1805,NaN,NaN
7950000US3603703,3603703,"NYC-Bronx Community District 10--Co-op City, P...",47551,584,47551,NaN,28927,1125,60.8,2.3,...,3.9,0.6,58739,3992,NaN,NaN,73666,2206,NaN,NaN


## Write to Database
Update list of variables and data types, build create table string, create datatable in temporary database.


In [20]:
dfexcel.replace({'dtype': {'int': 'INTEGER', 'float': 'REAL'}},inplace=True)
dfexcel.census_label.replace({'!!': ' - '},inplace=True, regex=True)
dfexcel.head()

,db_var,census_var,census_label,dtype
0,HSHD01_E,DP02_0001E,Estimate - HOUSEHOLDS BY TYPE - Total households,INTEGER
1,HSHD01_M,DP02_0001M,Margin of Error - HOUSEHOLDS BY TYPE - Total h...,INTEGER
2,HSHD01_PC,DP02_0001PE,Percent Estimate - HOUSEHOLDS BY TYPE - Total ...,INTEGER
3,HSHD01_PM,DP02_0001PM,Percent Margin of Error - HOUSEHOLDS BY TYPE -...,INTEGER
4,HSHD02_E,DP02_0002E,Estimate - HOUSEHOLDS BY TYPE - Total househol...,INTEGER


In [21]:
vardict=dfexcel.set_index('db_var').T.to_dict('list')
random.sample(vardict.items(), 2)

[('EDU04_M',
  ['DP02_0061M',
   'Margin of Error - EDUCATIONAL ATTAINMENT - Population 25 years and over - High school graduate (includes equivalency)',
   'INTEGER']),
 ('EMP05_PM',
  ['DP03_0005PM',
   'Percent Margin of Error - EMPLOYMENT STATUS - Population 16 years and over - In labor force - Civilian labor force - Unemployed',
   'REAL'])]

In [22]:
con = sqlite3.connect(dbname) 
cur = con.cursor()

In [23]:
cur.execute('DROP TABLE IF EXISTS {}'.format(tabname))
dbstring="""
CREATE TABLE {} (
GEOID TEXT,
GEOID2 TEXT NOT NULL PRIMARY KEY,
GEOLABEL TEXT,
""".format(tabname)

for k,v in vardict.items():
    dbstring=dbstring+k+' '+v[2]+', \n'
    
dbstring=dbstring[:-3]
dbstring=dbstring+');'
print(dbstring)


CREATE TABLE pumas_2017acs1 (
GEOID TEXT,
GEOID2 TEXT NOT NULL PRIMARY KEY,
GEOLABEL TEXT,
HSHD01_E INTEGER, 
HSHD01_M INTEGER, 
HSHD01_PC INTEGER, 
HSHD01_PM INTEGER, 
HSHD02_E INTEGER, 
HSHD02_M INTEGER, 
HSHD02_PC REAL, 
HSHD02_PM REAL, 
HSHD03_E INTEGER, 
HSHD03_M INTEGER, 
HSHD03_PC REAL, 
HSHD03_PM REAL, 
HSHD04_E REAL, 
HSHD04_M REAL, 
HSHD04_PC INTEGER, 
HSHD04_PM INTEGER, 
HSHD05_E REAL, 
HSHD05_M REAL, 
HSHD05_PC INTEGER, 
HSHD05_PM INTEGER, 
FERT01_E INTEGER, 
FERT01_M INTEGER, 
FERT01_PC INTEGER, 
FERT01_PM INTEGER, 
SCH01_E INTEGER, 
SCH01_M INTEGER, 
SCH01_PC INTEGER, 
SCH01_PM INTEGER, 
SCH02_E INTEGER, 
SCH02_M INTEGER, 
SCH02_PC REAL, 
SCH02_PM REAL, 
SCH03_E INTEGER, 
SCH03_M INTEGER, 
SCH03_PC REAL, 
SCH03_PM REAL, 
SCH04_E INTEGER, 
SCH04_M INTEGER, 
SCH04_PC REAL, 
SCH04_PM REAL, 
SCH05_E INTEGER, 
SCH05_M INTEGER, 
SCH05_PC REAL, 
SCH05_PM REAL, 
SCH06_E INTEGER, 
SCH06_M INTEGER, 
SCH06_PC REAL, 
SCH06_PM REAL, 
EDU01_E INTEGER, 
EDU01_M INTEGER, 
EDU01_PC INTEG

In [24]:
cur.execute(dbstring)

In [25]:
acsdata.to_sql(name=tabname, if_exists='append', index=True, con=con)

In [26]:
cur.execute('SELECT COUNT(*) FROM {};'.format(tabname))
rows = cur.fetchone()
print(rows[0], 'records written to', tabname)

55 records written to pumas_2017acs1


In [27]:
cur.execute('SELECT * FROM {} LIMIT 1;'.format(tabname))
col_names = [cn[0] for cn in cur.description]
print(len(col_names), 'columns written to', tabname)
#Number should be same as number in df acsdata plus 1, since index not included in df count

251 columns written to pumas_2017acs1


In [28]:
con.close()

## Metadata Table
DO NOT RERUN THIS SECTION FOR MULTIPLE GEOGRAPHIES. In the NYC Geodatabase there is only one metadata table for all of the ACS tables (acs1 and acs2) for all geographies. For whichever geography is processed first, set action variable to 'create' and run this entire series of blocks for the acs1 table. For the acs2 table, set the action variable to 'append' and skip the table creation and identifier insertion blocks.

In [ ]:
#Change table name and specify an action - you're creating the table for the first time with acs1 variables, 
#or appending the tables with acs2 variables

metatab='acslookup2017'
action='create' # 'create' or 'append'

In [ ]:
con = sqlite3.connect(dbname) 
cur = con.cursor()

In [ ]:
#Only run this block when creating initial table
if action=='create':
    mdstring="""
    CREATE TABLE {} (
    tabnum TEXT,
    est_id TEXT,
    est_value TEXT);
    """.format(metatab)
    cur.execute(mdstring)
else:
    print('Block not executed because "create" not selected as an action in earlier block')

In [ ]:
#Only run this block when creating initial table
if action=='create':
    exstring="""
        INSERT INTO {} VALUES('both','NOTE','Each variable has 4 values that are identified by a particular suffix: E for estimate, M for margin of error for the estimate, PC for percent total, and PM for margin of error for the percent total');
        INSERT INTO {} VALUES('both','GEOID','Id');
        INSERT INTO {} VALUES('both','GEOID2','Id2');
        INSERT INTO {} VALUES('both','GEOLABEL','Geography');
        """.format(metatab,metatab,metatab,metatab)
    cur.executescript(exstring)
    con.commit()
else:
    print('Block not executed because "create" not selected as an action in earlier block')

In [ ]:
#Run when creating table or when appending records
#Keys and values - db ids and labels - are simplified and truncated to 1 entry for each 4-column group (E,M,PC,PM)
if action in ('create','append'):
    for mk, mv in vardict.items():
        if mk.endswith('_E'):
            cur.execute("INSERT INTO {} values(?,?,?)".format(metatab),(worksheet,mk[:-2],mv[1][11:]))
    con.commit()
else:
    print('Block not executed because action not specified in earlier block')

In [ ]:
cur.execute('SELECT COUNT(*) FROM {};'.format(metatab))
rows = cur.fetchone()
print(rows[0], 'records in', metatab)

In [ ]:
action=''
con.close()